# **Homework 3 - Convolutional Neural Network**

This is the example code of homework 3 of the machine learning course by Prof. Hung-yi Lee.

In this homework, you are required to build a convolutional neural network for image classification, possibly with some advanced training tips.


There are three levels here:

**Easy**: Build a simple convolutional neural network as the baseline. (2 pts)

**Medium**: Design a better architecture or adopt different data augmentations to improve the performance. (2 pts)

**Hard**: Utilize provided unlabeled data to obtain better results. (2 pts)

## **About the Dataset**

The dataset used here is food-11, a collection of food images in 11 classes.

For the requirement in the homework, TAs slightly modified the data.
Please DO NOT access the original fully-labeled training data or testing labels.

Also, the modified dataset is for this course only, and any further distribution or commercial use is forbidden.

## **Import Packages**

First, we need to import packages that will be used later.

In this homework, we highly rely on **torchvision**, a library of PyTorch.

In [1]:
# Import necessary packages.
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
# "ConcatDataset" and "Subset" are possibly useful when doing semi-supervised learning.
from torch.utils.data import ConcatDataset, DataLoader, Subset, TensorDataset
from torchvision.datasets import DatasetFolder
import torchvision.models as models
# This is for the progress bar.
# from tqdm.auto import tqdm
torch.cuda.empty_cache()

## **Dataset, Data Loader, and Transforms**

Torchvision provides lots of useful utilities for image preprocessing, data wrapping as well as data augmentation.

Here, since our data are stored in folders by class labels, we can directly apply **torchvision.datasets.DatasetFolder** for wrapping data without much effort.

Please refer to [PyTorch official website](https://pytorch.org/vision/stable/transforms.html) for details about different transforms.

In [2]:
# It is important to do data augmentation in training.
# However, not every augmentation is useful.
# Please think about what kind of augmentation is helpful for food recognition.
train_tfm = transforms.Compose([
    # Resize the image into a fixed shape (height = width = 128)
    # transforms.Resize((128, 128)),
    transforms.RandomResizedCrop(size=(224, 224)),
    transforms.RandomPosterize(6),
    transforms.RandomAdjustSharpness(sharpness_factor=2),
    transforms.RandomAutocontrast(),
    transforms.RandomHorizontalFlip(),
    # You may add some transforms here.
    # ToTensor() should be the last one of the transforms.
    transforms.ToTensor(),
])

# We don't need augmentations in testing and validation.
# All we need here is to resize the PIL image and transform it into Tensor.
test_tfm = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])


In [3]:
from torch.utils.data import Dataset
class custom_subset(Dataset):
    r"""
    Subset of a dataset at specified indices.

    Arguments:
        dataset (Dataset): The whole Dataset
        indices (sequence): Indices in the whole set selected for subset
        labels(sequence) : targets as required for the indices. will be the same length as indices
    """
    def __init__(self, dataset, indices, labels):
        self.dataset = torch.utils.data.Subset(dataset, indices)
        self.targets = labels
        
    def __getitem__(self, idx):
        # print(self.dataset[idx])
        image = self.dataset[idx][0]
        target = self.targets[idx]
        
        return (image, target)

    def __len__(self):
        return len(self.targets)

In [4]:
# Batch size for training, validation, and testing.
# A greater batch size usually gives a more stable gradient.
# But the GPU memory is limited, so please adjust it carefully.
batch_size = 256

# Construct datasets.
# The argument "loader" tells how torchvision reads the data.
train_set = DatasetFolder("food-11/training/labeled", loader=lambda x: Image.open(x), extensions="jpg", transform=train_tfm)
valid_set = DatasetFolder("food-11/validation", loader=lambda x: Image.open(x), extensions="jpg", transform=test_tfm)
unlabeled_set = DatasetFolder("food-11/training/unlabeled", loader=lambda x: Image.open(x), extensions="jpg", transform=train_tfm)



# Construct data loaders.
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)


In [5]:
# print(train_set[0][0].shape)
# print(type(unlabeled_set))
print(len(train_set))
print(len(unlabeled_set))

3080
6786


## **Model**

The basic model here is simply a stack of convolutional layers followed by some fully-connected layers.

Since there are three channels for a color image (RGB), the input channels of the network must be three.
In each convolutional layer, typically the channels of inputs grow, while the height and width shrink (or remain unchanged, according to some hyperparameters like stride and padding).

Before fed into fully-connected layers, the feature map must be flattened into a single one-dimensional vector (for each image).
These features are then transformed by the fully-connected layers, and finally, we obtain the "logits" for each class.

### **WARNING -- You Must Know**
You are free to modify the model architecture here for further improvement.
However, if you want to use some well-known architectures such as ResNet50, please make sure **NOT** to load the pre-trained weights.
Using such pre-trained models is considered cheating and therefore you will be punished.
Similarly, it is your responsibility to make sure no pre-trained weights are used if you use **torch.hub** to load any modules.

For example, if you use ResNet-18 as your model:

model = torchvision.models.resnet18(pretrained=**False**) → This is fine.

model = torchvision.models.resnet18(pretrained=**True**)  → This is **NOT** allowed.

In [6]:
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        # The arguments for commonly used modules:
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        # torch.nn.MaxPool2d(kernel_size, stride, padding)

        # input image size: [3, 128, 128]
        self.cnn_layers = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),

            nn.Conv2d(64, 128, 3, 1, 1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),

            nn.Conv2d(128, 256, 3, 1, 1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(4, 4, 0),
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(256 * 8 * 8, 256),
            nn.ReLU(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Linear(256, 11)
        )

    def forward(self, x):
        # input (x): [batch_size, 3, 128, 128]
        # output: [batch_size, 11]

        # Extract features by convolutional layers.
        x = self.cnn_layers(x)

        # The extracted feature map must be flatten before going to fully-connected layers.
        x = x.flatten(1)

        # The features are transformed by fully-connected layers to obtain the final logits.
        x = self.fc_layers(x)
        return x

## **Training**

You can finish supervised learning by simply running the provided code without any modification.

The function "get_pseudo_labels" is used for semi-supervised learning.
It is expected to get better performance if you use unlabeled data for semi-supervised learning.
However, you have to implement the function on your own and need to adjust several hyperparameters manually.

For more details about semi-supervised learning, please refer to [Prof. Lee's slides](https://speech.ee.ntu.edu.tw/~tlkagk/courses/ML_2016/Lecture/semi%20(v3).pdf).

Again, please notice that utilizing external data (or pre-trained model) for training is **prohibited**.

In [7]:
class manual_shuffle_custom_subset(Dataset):
    r"""
    Subset of a dataset at specified indices.

    Arguments:
        dataset (Dataset): The whole Dataset
        indices (sequence): Indices in the whole set selected for subset
        labels(sequence) : targets as required for the indices. will be the same length as indices
    """
    def __init__(self, dataset):
        # self.dataset = torch.utils.data.Subset(dataset, indices)
        self.dataset = dataset
        # self.shuffle()

    def __getitem__(self, idx):
        image = self.dataset[idx][0]
        target = self.dataset[idx][1]
        return (image, target, idx)

    def __len__(self):
        return len(self.dataset)

    # def shuffle(self):
    #     self.seed = random.randint(0, 2**32 - 1)

    # def __iter__(self):
    #     n = len(self.dataset)
    #     indexes = list(range(n))
    #     random.Random(self.seed).shuffle(indexes)
    #     return iter(indexes)

In [8]:
candidate_idx = []
def get_pseudo_labels(dataset, model, threshold=0.4, is_balanced_sampling=True):
    # This functions generates pseudo-labels of a dataset using given model.
    # It returns an instance of DatasetFolder containing images whose prediction confidences exceed a given threshold.
    # You are NOT allowed to use any models trained on external data for pseudo-labeling.
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Construct a data loader.
    data_loader = DataLoader(manual_shuffle_custom_subset(dataset), batch_size=batch_size, shuffle=True)
    # print(dataset[:128])
    # Make sure the model is in eval mode.
    model.eval()
    # Define softmax function.
    softmax = nn.Softmax(dim=-1)
    # candidate_x = []
    # candidate_y = []
    
    new_label = []
    # Iterate over the dataset by batches.
    # for batch in tqdm(data_loader):
    
    for batch in data_loader:
        img, _, pseudo_set_idx = batch
        
        # Forward the data
        # Using torch.no_grad() accelerates the forward process.
        with torch.no_grad():
            # print(next(model.parameters()).is_cuda)
            logits = model(img.to(device))

        # Obtain the probability distributions by applying softmax on logits.
        probs = softmax(logits)
        # print(probs.argmax(dim=1))  torch.Size([128])
        # print(probs.argmax(dim=1))
        # print(probs.argmax(dim=1))
        # return
        for idx,max_val in enumerate(probs.argmax(dim=1)):
            
            if probs[idx,max_val]>threshold:
                if pseudo_set_idx[idx] not in candidate_idx:
                    # print(cnt*batch_size+idx,probs[idx,max_val])
                    candidate_idx.append(pseudo_set_idx[idx])
                    new_label.append(int(max_val.cpu())) 
                # print(cnt*batch_size+idx,dataset[cnt*batch_size+idx])
                # print(type(max_val_idx.cpu()))
        
        if is_balanced_sampling and len(new_label)>=len(train_set):
            break
                
        
        #     print(img[idx])
        #     print(img[idx].shape)
        # break
        # ---------- TODO ----------
        # Filter the data and construct a new dataset.

    # # Turn off the eval mode.
    model.train()
    
    # tensor_x = torch.stack(candidate_x) # a list of numpy arrays
    # tensor_y = torch.Tensor(candidate_y) # another list of numpy arrays (targets)
    # print(tensor_y)
    # my_dataset = [*zip(tensor_x, candidate_y)]
    new_dataset = custom_subset(dataset, candidate_idx, new_label)


    return new_dataset

In [9]:
# candidate_idx = []
# a = get_pseudo_labels(unlabeled_set,model,threshold=0.65)
# print(len(candidate_idx)/len(unlabeled_set))
# b=[1,2,3]
# c=torch.Tensor(b)
# c[1]
# np.array(b)
# print(type(b[0]))
# print(type(train_set[0][1]))

In [10]:
device = "cuda" if torch.cuda.is_available() else "cpu"
# model = models.resnet18(pretrained=False)

In [11]:
# print(model)

In [12]:
model = models.resnet18(pretrained=False)
num_ftrs = model.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model.fc = nn.Linear(num_ftrs, 11)

model = model.to(device)
# model.load_state_dict(torch.load("models/model_refined.ckpt"))
# model.load_state_dict(torch.load("models/model.ckpt"))
model.load_state_dict(torch.load("models/model_base.ckpt"))



<All keys matched successfully>

In [13]:
# print(model)

In [14]:
# model_classifier_temp = [*model.classifier][:-1]+[nn.Linear(in_features=4096, out_features=11, bias=True)]

# model.classifier = nn.Sequential(*model_classifier_temp)
# model = model.to(device)
# print(model)
# self.cnn_layers = nn.Sequential(
#             nn.Conv2d(3, 64, 3, 1, 1),
#             nn.BatchNorm2d(64),
#             nn.ReLU(),
#             nn.MaxPool2d(2, 2, 0),

#             nn.Conv2d(64, 128, 3, 1, 1),
#             nn.BatchNorm2d(128),
#             nn.ReLU(),
#             nn.MaxPool2d(2, 2, 0),

#             nn.Conv2d(128, 256, 3, 1, 1),
#             nn.BatchNorm2d(256),
#             nn.ReLU(),
#             nn.MaxPool2d(4, 4, 0),
#         )

In [15]:
# thresh = 0.4
# coeff_thresh = 0
# is_balanced_sampling = True
# epoch = 1
# candidate_idx = []
# pseudo_set = get_pseudo_labels(unlabeled_set, model, threshold=thresh-epoch*coeff_thresh, is_balanced_sampling=is_balanced_sampling) # threshold = (0.46-epoch*0.002)

In [16]:
# # pseudo_set[0][1]
# concat_dataset = ConcatDataset([train_set, pseudo_set])
# # concat_dataset[0][1]
# train_loader = DataLoader(concat_dataset, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
# for idx, i in enumerate(train_loader):
#     imgs, labels = i
#     print(idx)


In [13]:
# # "cuda" only when GPUs are available.
# device = "cuda" if torch.cuda.is_available() else "cpu"

# # Initialize a model, and put it on the device specified.
# model = models.inception_v3(pretrained=False).to(device)
# model.load_state_dict(torch.load("models/model.ckpt"))
# model.device = device

# For the classification task, we use cross-entropy as the measurement of performance.
criterion = nn.CrossEntropyLoss()

# Initialize optimizer, you may fine-tune some hyperparameters such as learning rate on your own.
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-5)

# The number of training epochs.
n_epochs = 160

# Whether to do semi-supervised learning.
do_semi = True
semi_epoch_interval = 5
thresh = 0.65
coeff_thresh = 0.003
is_balanced_sampling = False
early_stop = False


min_loss = 1000


for epoch in range(n_epochs):
    # ---------- TODO ----------
    # In each epoch, relabel the unlabeled dataset for semi-supervised learning.
    # Then you can combine the labeled dataset and pseudo-labeled dataset for the training.
    if do_semi and epoch%semi_epoch_interval==0:
        # Obtain pseudo-labels for unlabeled data using trained model.
        candidate_idx = []
        pseudo_set = get_pseudo_labels(unlabeled_set, model, threshold=thresh-epoch*coeff_thresh, is_balanced_sampling=is_balanced_sampling) # threshold = (0.46-epoch*0.002)
        if len(unlabeled_set)==len(candidate_idx):
            print("contain all unlabel dataset")
        elif 0.9<len(train_set)/len(pseudo_set)<1.1:
            print("concate dataset is balanced sampling")
        else:
            print("contain only {}%".format(len(candidate_idx)/len(unlabeled_set)))
        # Construct a new dataset and a data loader for training.
        # This is used in semi-supervised learning only.

        concat_dataset = ConcatDataset([train_set, pseudo_set])
        train_loader = DataLoader(concat_dataset, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
        # pseudo_train_loader = DataLoader(pseudo_set, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)

    # ---------- Training ----------
    # Make sure the model is in train mode before training.
    model.train()


    




    # if do_semi:
    #     pseudo_train_loss = []
    #     pseudo_train_accs = []
    #     total_pseudo_samples = 0
    #     for batch in pseudo_train_loader:
    #         if total_pseudo_samples>len(train_set) and is_balanced_sampling:
    #             break
    #         total_pseudo_samples += batch_size

    #         # A batch consists of image data and corresponding labels.
    #         imgs, labels = batch

    #         # Forward the data. (Make sure data and model are on the same device.)
    #         logits = model(imgs.to(device))

    #         # Calculate the cross-entropy loss.
    #         # We don't need to apply softmax before computing cross-entropy as it is done automatically.
    #         l2_coef = 0.001
    #         params_total = 0
    #         for i in model.parameters():
    #             params_total+=torch.norm(i)
            
    #         loss_semi = criterion(logits, labels.to(device))+l2_coef*params_total

    #         # Gradients stored in the parameters in the previous step should be cleared out first.
    #         optimizer.zero_grad()

    #         # Compute the gradients for parameters.
    #         loss_semi.backward()

    #         # Clip the gradient norms for stable training.
    #         grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)

    #         # Update the parameters with computed gradients.
    #         optimizer.step()

    #         # Compute the accuracy for current batch.
    #         acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

    #         # Record the loss and accuracy.
    #         pseudo_train_loss.append(loss_semi.item())
    #         pseudo_train_accs.append(acc)

    #     # The average loss and accuracy of the training set is the average of the recorded values.
    #     pseudo_train_loss = sum(pseudo_train_loss) / len(pseudo_train_loss)
    #     pseudo_train_accs = sum(pseudo_train_accs) / len(pseudo_train_accs)

    #     # Print the information.
    #     print(f"[ Pseudo Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {pseudo_train_loss:.5f}, acc = {pseudo_train_accs:.5f}")



    # These are used to record information in training.
    train_loss = []
    train_accs = []
    
    # Iterate the training set by batches.
    for batch in train_loader:

        # A batch consists of image data and corresponding labels.
        imgs, labels = batch

        # Forward the data. (Make sure data and model are on the same device.)
        logits = model(imgs.to(device))

        # Calculate the cross-entropy loss.
        # We don't need to apply softmax before computing cross-entropy as it is done automatically.
        l2_coef = 0.001
        params_total = 0
        for i in model.parameters():
            params_total+=torch.norm(i)
        loss = criterion(logits, labels.to(device))+l2_coef*params_total
        
        # Gradients stored in the parameters in the previous step should be cleared out first.
        optimizer.zero_grad()

        # Compute the gradients for parameters.
        loss.backward()

        # Clip the gradient norms for stable training.
        grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)

        # Update the parameters with computed gradients.
        optimizer.step()

        # Compute the accuracy for current batch.
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        # Record the loss and accuracy.
        train_loss.append(loss.item())
        train_accs.append(acc)

    # The average loss and accuracy of the training set is the average of the recorded values.
    train_loss = sum(train_loss) / len(train_loss)
    train_acc = sum(train_accs) / len(train_accs)

    # Print the information.
    print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")





    # ---------- Validation ----------
    # Make sure the model is in eval mode so that some modules like dropout are disabled and work normally.
    model.eval()

    # These are used to record information in validation.
    valid_loss = []
    valid_accs = []

    # Iterate the validation set by batches.
    for batch in valid_loader:

        # A batch consists of image data and corresponding labels.
        imgs, labels = batch

        # We don't need gradient in validation.
        # Using torch.no_grad() accelerates the forward process.
        with torch.no_grad():
          logits = model(imgs.to(device))

        # We can still compute the loss (but not the gradient).
        loss = criterion(logits, labels.to(device))

        # Compute the accuracy for current batch.
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        # Record the loss and accuracy.
        valid_loss.append(loss.item())
        valid_accs.append(acc)

    # The average loss and accuracy for entire validation set is the average of the recorded values.
    valid_loss = sum(valid_loss) / len(valid_loss)
    valid_acc = sum(valid_accs) / len(valid_accs)

    # Print the information.
    print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")
    print("-------------------finish epoch--------------------------------")
    
    
    if valid_loss < min_loss:
        # Save model if your model improved
        min_loss = valid_loss
        print('Saving model (epoch = {:4d}, loss = {:.4f})'
            .format(epoch + 1, min_loss))
        torch.save(model.state_dict(), 'models/model_refined.ckpt')  # Save model to specified path
        early_stop_cnt = 0
    else:
        early_stop_cnt += 1
    if early_stop and early_stop_cnt>50:
        break

contain only 0.29089301503094606%
[ Train | 001/160 ] loss = 1.81495, acc = 0.62517
[ Valid | 001/160 ] loss = 1.69532, acc = 0.46780
-------------------finish epoch--------------------------------
Saving model (epoch =    1, loss = 1.6953)
[ Train | 002/160 ] loss = 1.75030, acc = 0.63899
[ Valid | 002/160 ] loss = 1.68908, acc = 0.45925
-------------------finish epoch--------------------------------
Saving model (epoch =    2, loss = 1.6891)
[ Train | 003/160 ] loss = 1.71357, acc = 0.64838
[ Valid | 003/160 ] loss = 1.71755, acc = 0.45207
-------------------finish epoch--------------------------------
[ Train | 004/160 ] loss = 1.72145, acc = 0.64441
[ Valid | 004/160 ] loss = 1.77230, acc = 0.44303
-------------------finish epoch--------------------------------
[ Train | 005/160 ] loss = 1.66295, acc = 0.66291
[ Valid | 005/160 ] loss = 1.75542, acc = 0.43296
-------------------finish epoch--------------------------------
contain only 0.40819333922782197%
[ Train | 006/160 ] loss =

## **Testing**

For inference, we need to make sure the model is in eval mode, and the order of the dataset should not be shuffled ("shuffle=False" in test_loader).

Last but not least, don't forget to save the predictions into a single CSV file.
The format of CSV file should follow the rules mentioned in the slides.

### **WARNING -- Keep in Mind**

Cheating includes but not limited to:
1.   using testing labels,
2.   submitting results to previous Kaggle competitions,
3.   sharing predictions with others,
4.   copying codes from any creatures on Earth,
5.   asking other people to do it for you.

Any violations bring you punishments from getting a discount on the final grade to failing the course.

It is your responsibility to check whether your code violates the rules.
When citing codes from the Internet, you should know what these codes exactly do.
You will **NOT** be tolerated if you break the rule and claim you don't know what these codes do.


In [ ]:
# Make sure the model is in eval mode.
# Some modules like Dropout or BatchNorm affect if the model is in training mode.
# model = Classifier().to(device)
model.load_state_dict(torch.load("models/model.ckpt"))
model.eval()
test_set = DatasetFolder("food-11/testing", loader=lambda x: Image.open(x), extensions="jpg", transform=test_tfm)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)
# Initialize a list to store the predictions.
predictions = []

# Iterate the testing set by batches.
for batch in test_loader:
    # A batch consists of image data and corresponding labels.
    # But here the variable "labels" is useless since we do not have the ground-truth.
    # If printing out the labels, you will find that it is always 0.
    # This is because the wrapper (DatasetFolder) returns images and labels for each batch,
    # so we have to create fake labels to make it work normally.
    imgs, labels = batch

    # We don't need gradient in testing, and we don't even have labels to compute loss.
    # Using torch.no_grad() accelerates the forward process.
    with torch.no_grad():
        logits = model(imgs.to(device))
    
    # Take the class with greatest logit as prediction and record it.
    predictions.extend(logits.argmax(dim=-1).cpu().numpy().tolist())

In [ ]:
# Save predictions into the file.
with open("predict.csv", "w") as f:

    # The first row must be "Id, Category"
    f.write("Id,Category\n")

    # For the rest of the rows, each image id corresponds to a predicted class.
    for i, pred in  enumerate(predictions):
         f.write(f"{i},{pred}\n")